In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
project_dir = '/content/drive/My Drive/cnn-dailymail-summarizer'
os.chdir(project_dir)

!pip install -r requirements.txt

In [ ]:

import pandas as pd
from cnn_dailymail_news_text_summarizer.dataset import load_datasets, remove_punctuation, preprocess_text, tokenize, save_tokenized_datasets, load_tokenized_datasets
from cnn_dailymail_news_text_summarizer.plots import plot_num_characters, plot_num_words, plot_num_sentences, plot_mean_word_length, create_corpus, plot_most_frequent_stopwords, plot_most_frequent_words, get_top_ngram
from cnn_dailymail_news_text_summarizer.training import train_model
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import nltk
from collections import Counter
from collections import defaultdict
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, concatenate_datasets
import torch
import evaluate

In [ ]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

## Loading Data

In [ ]:

train_path = os.path.join(project_dir, 'data/raw/cnn_dailymail/train.csv')
test_path = os.path.join(project_dir, 'data/raw/cnn_dailymail/test.csv')
val_path = os.path.join(project_dir, 'data/raw/cnn_dailymail/validation.csv')


In [ ]:
train_data, test_data, val_data = load_datasets(train_path, test_path, val_path)

In [ ]:
train_data.head()

## Exploratory Data Analysis



In [ ]:
sample = train_data.sample()
list(sample['article'])

In [ ]:
list(sample['highlights'])

In [ ]:
len(train_data)

### Counts and Lengths

In [ ]:
eda_data = train_data.sample(frac=0.1)

In [ ]:
plot_num_characters(eda_data, 'article')

In [ ]:
plot_num_words(eda_data, 'article')

In [ ]:
nltk.download('punkt')

In [ ]:
plot_num_sentences(eda_data, 'article')

In [ ]:
plot_mean_word_length(eda_data, 'article')

In [ ]:
plot_num_characters(eda_data, 'highlights')

In [ ]:
plot_num_words(eda_data, 'highlights')

In [ ]:
plot_num_sentences(eda_data, 'highlights')

### Term frequency

In [ ]:
eda_data.drop('mean_word_length', axis=1, inplace=True)

In [ ]:
nltk.download('stopwords')

In [ ]:
stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
plot_most_frequent_stopwords(eda_data, stop)

In [ ]:
plot_most_frequent_words(eda_data, stop)

### N-gram frequency

In [ ]:
eda_data = eda_data.sample(frac=0.1)

In [ ]:
get_top_ngram(eda_data, list(stop), 2)

In [ ]:
get_top_ngram(eda_data, list(stop), 3)

In [ ]:
get_top_ngram(eda_data, list(stop), 5)

## Data Preprocessing

In [ ]:
checkpoint = "facebook/bart-base"

In [ ]:
tokenizer = BartTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer(train_data['article'][0])

In [ ]:
sample = test_data.sample()
sample_text = sample['article'].iloc[0]
print(sample_text)

In [ ]:
model = BartForConditionalGeneration.from_pretrained(checkpoint)
model.to(device_name)
inputs = tokenizer(sample_text, max_length=1024, return_tensors='pt', truncation=True)
inputs = {key: value.to(device_name) for key, value in inputs.items()}
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Predicted summary: ", summary)
print("")
print("Actual summary: ", sample['highlights'].iloc[0])

In [ ]:
#train_data = train_data.sample(frac=0.2, random_state=42)
#val_data = val_data.sample(frac=0.2, random_state=42)
#test_data = test_data.sample(frac=0.2, random_state=42)

In [ ]:
#train_data = tokenize(train_data, tokenizer)

In [ ]:
#print(train_data)

In [ ]:
#val_data = tokenize(val_data, tokenizer)

In [ ]:
#test_data = tokenize(test_data, tokenizer)

In [ ]:
processed_path = os.path.join(project_dir, 'data/processed/')


In [ ]:
#save_tokenized_datasets(train_data, test_data, val_data, processed_path)
train_data, _, val_data = load_tokenized_datasets(processed_path)

## Fine-tuning

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, padding=True)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./model',
    num_train_epochs=11,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True,
    fp16=True,
    remove_unused_columns=True,
    save_safetensors=False
)


In [ ]:
train_model(train_data, val_data, tokenizer, data_collator, training_args, device_name, rouge, checkpoint=True)

In [ ]:
torch.cuda.empty_cache()

## Evaluation

In [ ]:
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/cnn-dailymail-summarizer/model/checkpoint-9867')
model.to('cuda')

In [ ]:
sample = test_data.sample()
sample_text = sample['article'].iloc[0]
print(sample_text)

In [ ]:
inputs = tokenizer(sample_text, max_length=1024, return_tensors='pt', truncation=True)
inputs = {key: value.to(device_name) for key, value in inputs.items()}
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Predicted summary: ", summary)
print("")
print("Actual summary: ", sample['highlights'].iloc[0])